In [1]:
# Importing required libraries
import pandas as pd
import re
import requests
from io import BytesIO
import numpy as np
from typing import Optional

In [2]:
# Step 1: Loading the HIV 2015 ORS dataset
df = pd.read_csv("../data/HIV2015_ORS.csv")


In [3]:
# Checking dataset shape and columns

print("Shape:", df.shape)

print("\nFirst 30 columns:")
print(df.columns[:30])

print("\nSample rows:")

# Previewing the first few rows
df.head(3)


Shape: (237, 84)

First 30 columns:
Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       '1', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17',
       'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21',
       'Unnamed: 22', 'Unnamed: 23', 'Impact Scores', 'Unnamed: 25',
       'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29'],
      dtype='object')

Sample rows:


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,1,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,"Used in Col L, N",Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,OVERALL,OVERALL,OVERALL,OVERALL
1,Country,WHO Region,Population,Geographical Region,WHO Group,DALY,Adult DALYs,Children DALYs,Retention Rate,Retention Rate (ADULT),...,All ages,Children (0-14),Adults (15+),Year,NaN,http://apps.who.int/gho/data/node.main.626?lan...,Estimated antiretroviral therapy coverage amon...,Reported number of people receiving antiretrov...,Cleaned coverage,Cleaned number of people receiving antiretrovi...
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,max of RR and 97.14 (1/35),max of RR and 97.14 (1/35),...,72,60,73,2015,NaN,Country,2015,2015,NaN,NaN


In [4]:
# Step 2: Fixing multi-row headers
# The real column names are not in the first row


new_header = df.iloc[1]    # Row that contains actual column names
df = df.iloc[3:]           # Remove title and metadata rows
df.columns = new_header    # Assign correct column names
df = df.reset_index(drop=True)

df.head(3)


1,Country,WHO Region,Population,Geographical Region,WHO Group,DALY,Adult DALYs,Children DALYs,Retention Rate,Retention Rate (ADULT),...,All ages,Children (0-14),Adults (15+),Year,NaN,http://apps.who.int/gho/data/node.main.626?lang=en,Estimated antiretroviral therapy coverage among people living with HIV (%),Reported number of people receiving antiretroviral therapy,Cleaned coverage,Cleaned number of people receiving antiretroviral therapy
0,Afghanistan,EMR,"33,736,494.00","East, South and South-East Asia",A,"10,752.55","9,224.37","1,528.18",72.00,73.00,...,92,100,92,2015,NaN,Afghanistan,5 [3-12],364,5.00%,364.00
1,Albania,EUR,"2,880,703.00",Europe and Central Asia,A,98.50,96.60,1.90,92.00,92.00,...,92,77,92,2015,NaN,Albania,No data,423,NaN,423.00
2,Algeria,AFR,"39,871,528.00",Middle East and North Africa,A,"11,586.04","11,055.12",530.92,92.00,92.00,...,100,NaN,NaN,2015,NaN,Algeria,90 [70->95],7 915,90.00%,"7,915.00"


In [5]:
# Step 3: Exploring Impact-related columns

[c for c in df.columns if "Impact" in str(c)]


['Overall Treatment Impact', 'Final Impact']

In [6]:
# Looking at last few columns just to understand the structure

list(df.columns[-40:])


['Treatment Regimen Proportion and Efficacy Section',
 nan,
 nan,
 nan,
 nan,
 'impact =daly*treatment cover*x/(1-treatment cover*x)/number of drugs',
 nan,
 nan,
 nan,
 nan,
 nan,
 'Originator Company',
 'Drug(s)',
 'Final Impact',
 nan,
 nan,
 nan,
 nan,
 'Afghanistan',
 '9,224.37',
 'Afghanistan',
 '1,528.18',
 nan,
 'Afghanistan',
 '500',
 '26',
 '11.00%',
 nan,
 nan,
 'Country',
 'All ages',
 'Children (0-14)',
 'Adults (15+)',
 'Year',
 nan,
 'http://apps.who.int/gho/data/node.main.626?lang=en',
 'Estimated antiretroviral therapy coverage among people living with HIV (%)',
 'Reported number of people receiving antiretroviral therapy',
 'Cleaned coverage',
 'Cleaned number of people receiving antiretroviral therapy']

In [7]:
# Step 4: Identifying drug-related columns using keywords

keywords = ["AZT", "TDF", "EFV", "3TC", "NVP", "LPV", "ABC", "FTC", "DRV", "ATV", "DTG"]
[c for c in df.columns if any(k in str(c).upper() for k in keywords)]


['3TC', 'ABC', 'AZT', 'EFV', 'FTC', 'LPV/r', 'NVP', 'TDF', 'ATV/r']

In [8]:
# Step 5: Detecting columns that contain regimen strings with "+"

cols_with_plus = []

for idx in range(df.shape[1]):
    col_series = df.iloc[:, idx]  
    if col_series.dtype == "object":
        sample = col_series.dropna().astype(str).head(80)
        if sample.str.contains(r"\+").any():
            cols_with_plus.append(df.columns[idx])

cols_with_plus



['Treatment Regimen Proportion and Efficacy Section']

In [9]:
# Step 6: Make column names unique

import numpy as np
import pandas as pd

def make_unique(cols):
    seen = {}
    out = []
    for c in cols:
        if c is None or (isinstance(c, float) and np.isnan(c)):
            c = "__nan__"
        c = str(c).strip()
        if c in seen:
            seen[c] += 1
            out.append(f"{c}.{seen[c]}")
        else:
            seen[c] = 0
            out.append(c)
    return out

df.columns = make_unique(df.columns)


In [10]:
# Step 7: Selecting drug columns safely

possible_drugs = ['3TC', 'ABC', 'AZT', 'EFV', 'FTC', 'LPV/r', 'NVP', 'TDF', 'ATV/r']
drug_cols = [c for c in possible_drugs if c in df.columns]

print("Drug cols found:", drug_cols)


# Step 8: Converting drug and impact columns to numeric
def to_num(s):
    return pd.to_numeric(s.astype(str).str.replace(",", ""), errors="coerce")


for c in drug_cols + ['Overall Treatment Impact', 'Final Impact']:
    if c in df.columns:
        df[c] = to_num(df[c])


Drug cols found: ['3TC', 'ABC', 'AZT', 'EFV', 'FTC', 'LPV/r', 'NVP', 'TDF', 'ATV/r']


In [11]:
# Step 9: Computing overall treatment impact per country

out = df[['Country'] + drug_cols].copy()

out['Computed Overall Treatment Impact'] = out[drug_cols].sum(axis=1)

# Step 10: Comparing with provided impact values (if available)

if 'Overall Treatment Impact' in df.columns:
    out['Given Overall Treatment Impact'] = df['Overall Treatment Impact']
    out['Diff (Computed - Given)'] = out['Computed Overall Treatment Impact'] - out['Given Overall Treatment Impact']

out.head(5)

if 'Diff (Computed - Given)' in out.columns:
    print("Max abs diff:", out['Diff (Computed - Given)'].abs().max())



Max abs diff: 3825.9700000000885


In [12]:
# Step 11: Saving the final output

import os
os.makedirs("../output", exist_ok=True)

out.to_csv("../output/impact_score.csv", index=False)
print("Saved to output/impact_score.csv")


Saved to output/impact_score.csv
